In [4]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

In [ ]:
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
#overview of the data
DATA_FILE_PATH = os.path.join(os.getcwd(),'data','100_50')

dir_list = os.listdir(DATA_FILE_PATH)
max_data_points = len(dir_list)
print("Number of data points: {}".format(len(dir_list)))
print(dir_list[0])


Number of data points: 3129
Agent1_485135


<h1>Data information:</h1>

All the data will bes stored in a pandas data frame

Three primary datasets can be retreived from the data in the folder
<ul>
    <li>Iterative method data</li>
    <li>End to End data</li>
    <li>Compliance clasification data</li>
</ul>

In [ ]:
def unpackLoadCondtions(loadConditions):
    forces = loadConditions['a']
    free = loadConditions['b']
    passive = loadConditions['c']
    formating_array = loadConditions['d']

    volfrac = formating_array[0]
    nelx = int(formating_array[1])
    nely = int(formating_array[2])
    penal = formating_array[3]
    rmin = formating_array[4]

    #print(volfrac,nelx,nely)

    return forces,free,passive,volfrac,nelx,nely,penal,rmin


def unpackIteration(iteration):
    x = iteration['a']
    xPhys = iteration['b']
    formating_array = iteration['c']

    compliance = formating_array[0]
    change = formating_array[1]
    mass = formating_array[2]

    #print(compliance,change,mass)
    return x,xPhys,compliance,change,mass
